In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 82% 43.0M/52.6M [00:01<00:00, 58.1MB/s]
100% 52.6M/52.6M [00:01<00:00, 48.8MB/s]


In [ ]:
# extracting the compessed dataset
from zipfile import ZipFile
dataset = '/content/jigsaw-toxic-comment-classification-challenge.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
!ls

jigsaw-toxic-comment-classification-challenge.zip  sample_submission.csv.zip  train.csv.zip
kaggle.json					   test.csv.zip
sample_data					   test_labels.csv.zip


In [10]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [12]:
df = pd.read_csv(os.path.join('/content/jigsaw-toxic-comment-classification-challenge.zip', '/content/train.csv.zip', '/content/train.csv.zip'))
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
